# 3. Manejando Netcdf

Para manejar netcdfs usando python haremos uso extensivo de los paquetes estrella de la plataforma Pangeo. Pangeo es un esfuerzo de la comunidad que promueve la reproductibilidad, escalamiento y transparencia en la ciencia.

El paquete de cabecera para el manejo de netcdfs será [xarray](http://xarray.pydata.org/en/latest/). Xarray extiende las capacidades de numpy en el manejo de datos n-dimensionales introduciendo etiquetas en forma de dimensiones, coordenadas y atributos que facilitan las operaciones sobre los mismos. El motor para las operaciones entre fechas es proporcionado por pandas y la escalabilidad de los datos es proporcionado por dask, lo cual convierte a este paquete en una herramienta poderosa.

In [ ]:
%matplotlib inline
import numpy as np
import xarray as xr

## 3.1 Estructuras de xarray

xarray cuenta con dos estructuras simples que son fundamentales comprender para un mejor uso del paquete. Esta es una brevisima descripción sobre algunos conceptos claves que se deben tener presentes, mayor informacion la pueden encontrar siempre en la [documentacion](http://xarray.pydata.org/en/stable/data-structures.html#)

### 3.1.1 DataArray

Un `DataArray` es virtualmente igual a un arreglo n-dimensional de numpy que cuenta con etiquetas en sus dimensiones, siendo el elemento base de xarray. Para denifinir un `DataArray` necesitaremos proporcionar, como mínimo, una arreglo de elementos numéricos.

In [ ]:
xr.DataArray(np.random.randn(10,40))

Como podemos observar, xarray automaticamente asignó nombres a las dimensiones y nos informa que no encuentra coordenadas asociadas a estas dimensiones. Ahora procederemos a declarar un `DataArray` con la información que falta

In [ ]:
# Creamos un poco de data falsa
lat = np.arange(-90, 90, 0.25)
lon = np.arange(0, 360, 0.25)
llon, llat = np.meshgrid(lon, lat)
data = np.sin(llat**2-llon**2)

# Creamos el DataArray
xarr = xr.DataArray(data, coords=[lat,lon], dims=["lat","lon"])
xarr

Nuestro `DataArray` ha sido creado dentro de la variable `xarr` satisfactoriamente con las dimensiones y coordenadas asignadas correctamente. Ahora podemos acceder a los métodos que xarray ofrece para sus objetos.

In [ ]:
xarr.plot()

### 3.1.2 Dataset

Un `Dataset` es la representación en la memoria del sistema de un archivo netcdf. Un `Dataset` es un conjunto de `DataArray`

## 3.2 Datos Reales

Para cargar datos en formato netcdf a una variable, se hacer uso de la función `open_dataset` (en el caso de tener 1 solo archivo) o `open_mfdataset` en caso de tener muchos archivos

In [ ]:
ersstv5 = xr.open_dataset('https://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/noaa.ersst.v5/sst.mnmean.nc')
ersstv5

In [ ]:
ersstv5.sst

In [ ]:
iri_errst.sst.isel(zlev=0,T=-1).plot()